In [200]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE


In [201]:
os.chdir("C:/Users/NIshant/Downloads")

In [202]:
train=pd.read_csv("train_ZoGVYWq.csv")

In [203]:
test=pd.read_csv("test_66516Ee.csv")

In [215]:
df_train.head()

,perc_premium_paid_by_cash_credit,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,no_of_premiums_paid,sourcing_channel,residence_area_type,premium,age_years
0,0.429,355060,0.0,0.0,0.0,13,1,1,3300,1.0
1,0.010,315150,0.0,0.0,0.0,21,0,1,18000,0.0
2,0.917,84140,2.0,3.0,1.0,7,1,0,3300,0.0
3,0.049,250510,0.0,0.0,0.0,9,0,1,9600,0.0
4,0.052,198680,0.0,0.0,0.0,12,0,1,9600,NaN


In [ ]:
test.head()

In [204]:
df=train
df['Count_3-6_months_late'].fillna(df['Count_3-6_months_late'].median(),inplace=True)
df['Count_6-12_months_late'].fillna(df['Count_6-12_months_late'].median(),inplace=True)
df['Count_more_than_12_months_late'].fillna(df['Count_more_than_12_months_late'].median(),inplace=True)
df['application_underwriting_score'].fillna(df['Count_more_than_12_months_late'].median(),inplace=True)

In [ ]:
df['sourcing_channel'] = df['sourcing_channel'].replace({'A':0, 'B':0,'C':1,'D':0,'E':0})

In [205]:
test['Count_3-6_months_late'].fillna(df['Count_3-6_months_late'].median(),inplace=True)
test['Count_6-12_months_late'].fillna(df['Count_6-12_months_late'].median(),inplace=True)
test['Count_more_than_12_months_late'].fillna(df['Count_more_than_12_months_late'].median(),inplace=True)
test['application_underwriting_score'].fillna(df['Count_more_than_12_months_late'].median(),inplace=True)

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='perc_premium_paid_by_cash_credit')

In [ ]:

sns.factorplot('renewal',data=df,kind='count',hue='no_of_premiums_paid')

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='sourcing_channel')

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='residence_area_type')

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='premium')

In [ ]:
sns.factorplot('renewal',data=df_train,kind='count',hue='age_years')

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='Count_3-6_months_late')

In [ ]:
sns.factorplot('renewal',data=df,kind='count',hue='Count_6-12_months_late')


In [ ]:

sns.factorplot('renewal',data=df,kind='count',hue='Count_more_than_12_months_late')


In [ ]:
df['age_years'].hist(bins=70)

In [ ]:
df.groupby(['age_years']).renewal.mean()

In [ ]:
df.groupby(['residence_area_type']).renewal.mean()

In [206]:
df_train=df
df_renewed = df.renewal
df_train= df_train.drop('renewal',axis=1)
df_train= df_train.drop('id',axis=1)
df_train= df_train.drop('application_underwriting_score',axis=1)




In [ ]:
df_train.head()

In [ ]:
test= test.drop('id',axis=1)
test= test.drop('application_underwriting_score',axis=1)


In [207]:
df_train['residence_area_type'].replace(['Rural','Urban'],[0,1],inplace=True)
test['residence_area_type'].replace(['Rural','Urban'],[0,1],inplace=True)

In [ ]:
df_train['sourcing_channel'].replace(['A','B','C','D','E'],[0,1,2,3,4],inplace=True)
test['sourcing_channel'].replace(['A','B','C','D','E'],[0,1,2,3,4],inplace=True)

In [208]:
df_train['sourcing_channel'] = df_train['sourcing_channel'].replace({'A':0, 'B':0,'C':1,'D':0,'E':0})
test['sourcing_channel'] = test['sourcing_channel'].replace({'A':0, 'B':0,'C':1,'D':0,'E':0})

In [209]:
df_train['age_years']=df_train['age_in_days']/365
categorical_features = ['age_in_days']
df_train = df_train.drop(categorical_features, axis=1)
test['age_years']=test['age_in_days']/365
categorical_features = ['age_in_days']
test = test.drop(categorical_features, axis=1)

In [ ]:
df_train.head()

In [210]:
age_labels = ['20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
df_train['age_years'] = pd.cut(df_train.age_years, range(20, 81, 10), right=False, labels=age_labels)
test['age_years'] = pd.cut(test.age_years, range(20, 81, 10), right=False, labels=age_labels)

In [211]:
df_train['age_years'] = df_train['age_years'].replace({'20-29':2, '30-39':1, '40-49':0, '50-59':0, '60-69':1, '70-79':2})
test['age_years'] = test['age_years'].replace({'20-29':2, '30-39':1, '40-49':0, '50-59':0, '60-69':1, '70-79':2})

In [ ]:
df_train= df_train.drop('age_in_days',axis=1)

In [222]:
X_train, X_test, Y_train, Y_test = train_test_split(df_train,df_renewed,test_size=0.2)

In [ ]:
X_test.head()

In [213]:
ss = StandardScaler()
scale_features = ['Income']
df[scale_features] = ss.fit_transform(df[scale_features])
test[scale_features] = ss.fit_transform(test[scale_features])

In [214]:
ss = StandardScaler()
scale_features = ['premium']
df[scale_features] = ss.fit_transform(df[scale_features])
test[scale_features] = ss.fit_transform(test[scale_features])

In [ ]:
ss = StandardScaler()
scale_features = ['age_in_days']
df[scale_features] = ss.fit_transform(df[scale_features])
test[scale_features] = ss.fit_transform(test[scale_features])

In [ ]:
ss = StandardScaler()
scale_features = ['application_underwriting_score']
df[scale_features] = ss.fit_transform(df[scale_features])
test[scale_features] = ss.fit_transform(test[scale_features])

In [ ]:
ss = StandardScaler()
scale_features = ['no_of_premiums_paid']
df[scale_features] = ss.fit_transform(df[scale_features])
test[scale_features] = ss.fit_transform(test[scale_features])

In [ ]:
#oversampler=SMOTE(random_state=0)
#os_features,os_labels=oversampler.fit_sample(X_train,Y_train)

In [231]:
sm = SMOTE(random_state=0)

In [232]:
X_resampled_train, y_resampled_train = sm.fit_sample(X_train, Y_train)

In [233]:
X_train.head()

,perc_premium_paid_by_cash_credit,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,no_of_premiums_paid,sourcing_channel,residence_area_type,premium,age_years
46591,0.121,371300,0.0,0.0,0.0,16,1,1,41100,1.0
41059,0.639,60050,0.0,0.0,0.0,7,0,1,3300,2.0
59317,0.837,60070,0.0,0.0,0.0,6,0,1,3300,2.0
56538,0.274,165580,0.0,0.0,0.0,8,0,1,15900,1.0
66374,0.194,292550,0.0,0.0,0.0,7,0,1,7500,0.0


In [235]:
X_resampled_train=pd.DataFrame(X_resampled_train,columns=['perc_premium_paid_by_cash_credit','Income','Count_3-6_months_late','Count_6-12_months_late','Count_more_than_12_months_late','no_of_premiums_paid','sourcing_channel','residence_area_type','premium','age_years'])
y_resampled_train=pd.DataFrame(y_resampled_train,columns=['renewal'])

In [220]:
X_train['age_years'].fillna(X_train['age_years'].median(),inplace=True)

In [219]:
df_train.head()

,perc_premium_paid_by_cash_credit,Income,Count_3-6_months_late,Count_6-12_months_late,Count_more_than_12_months_late,no_of_premiums_paid,sourcing_channel,residence_area_type,premium,age_years
0,0.429,355060,0.0,0.0,0.0,13,1,1,3300,1.0
1,0.010,315150,0.0,0.0,0.0,21,0,1,18000,0.0
2,0.917,84140,2.0,3.0,1.0,7,1,0,3300,0.0
3,0.049,250510,0.0,0.0,0.0,9,0,1,9600,0.0
4,0.052,198680,0.0,0.0,0.0,12,0,1,9600,1.0


In [252]:
columns=['perc_premium_paid_by_cash_credit','Income','Count_3-6_months_late','Count_6-12_months_late','Count_more_than_12_months_late','no_of_premiums_paid','sourcing_channel','residence_area_type','premium','age_years']
#decision_tree = LogisticRegression()
#decision_tree = GaussianNB()
decision_tree = XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
#decision_tree = RandomForestClassifier()
#decision_tree = SVC()
# Fit to the training data
#test_df = test_df[train_df.columns]
#decision_tree.fit(X_resampled_train,y_resampled_train)
decision_tree.fit(X_train, Y_train)
#Y_pred2 = decision_tree.predict(X_test)
Y_pred2 = decision_tree.predict_proba(test[columns])[:,1]
decision_tree.score(X_train, Y_train)

C:\Users\NIshant\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.958454650762343

In [253]:
metrics.confusion_matrix(Y_test, Y_pred2)

ValueError: Found input variables with inconsistent numbers of samples: [15971, 34224]

In [254]:
submission = pd.DataFrame({
        "renewal": Y_pred2
    })
submission.to_csv("submission.csv",index=False)